In [1]:
import simpy
import random

# Define constants based on the provided parameters
NUM_CLIENTS = random.randint(100, 200)
NUM_DATACENTERS = 1
NUM_HOSTS = 2

# Cloud node parameters
CLOUD_NODES = 10
CLOUD_CPU_POWER = (3000, 5000)  # MIPS
CLOUD_RAM = 8  # GB
CLOUD_STORAGE = 1  # TB
CLOUD_BANDWIDTH = 0.5  # Gb/s

# Fog node parameters
FOG_NODES = 10
FOG_CPU_POWER = (1000, 2800)  # MIPS
FOG_RAM = 2  # GB
FOG_STORAGE = 0.5  # TB
FOG_BANDWIDTH = 1  # Gb/s

# Task parameters
TASK_LENGTH = (2000, 56000)  # MI
DATA_SIZE = (400, 600)  # MB
NUM_TASKS = random.randint(300, 1500)

class Node:
    def __init__(self, env, cpu_power, ram, storage, bandwidth):
        self.env = env
        self.cpu_power = cpu_power
        self.ram = ram
        self.storage = storage
        self.bandwidth = bandwidth
        self.queue = simpy.Resource(env, capacity=1)

    def process_task(self, task_length, data_size):
        # Processing time is based on task length and CPU power
        processing_time = task_length / self.cpu_power
        yield self.env.timeout(processing_time)

def generate_tasks(env, nodes, task_length_range, data_size_range, num_tasks):
    for _ in range(num_tasks):
        task_length = random.randint(*task_length_range)
        data_size = random.randint(*data_size_range)
        node = random.choice(nodes)
        env.process(handle_task(env, node, task_length, data_size))
        yield env.timeout(1)  # Small delay to simulate task arrival interval

def handle_task(env, node, task_length, data_size):
    with node.queue.request() as request:
        yield request
        print(f"Task assigned to node with CPU power {node.cpu_power} at time {env.now}")
        yield env.process(node.process_task(task_length, data_size))
        print(f"Task completed at time {env.now}")

def main():
    env = simpy.Environment()

    # Create cloud nodes
    cloud_nodes = [
        Node(env, random.randint(*CLOUD_CPU_POWER), CLOUD_RAM, CLOUD_STORAGE, CLOUD_BANDWIDTH)
        for _ in range(CLOUD_NODES)
    ]

    # Create fog nodes
    fog_nodes = [
        Node(env, random.randint(*FOG_CPU_POWER), FOG_RAM, FOG_STORAGE, FOG_BANDWIDTH)
        for _ in range(FOG_NODES)
    ]

    all_nodes = cloud_nodes + fog_nodes

    # Generate and process tasks
    env.process(generate_tasks(env, all_nodes, TASK_LENGTH, DATA_SIZE, NUM_TASKS))

    # Run the simulation
    env.run()

if __name__ == "__main__":
    main()


Task assigned to node with CPU power 1674 at time 0
Task assigned to node with CPU power 4926 at time 1
Task assigned to node with CPU power 2599 at time 2
Task assigned to node with CPU power 1859 at time 3
Task assigned to node with CPU power 1224 at time 4
Task completed at time 4.826856483262794
Task assigned to node with CPU power 1451 at time 5
Task assigned to node with CPU power 2137 at time 6
Task assigned to node with CPU power 3604 at time 11
Task completed at time 11.098660170523752
Task assigned to node with CPU power 1469 at time 12
Task completed at time 12.366188769414576
Task assigned to node with CPU power 1674 at time 12.366188769414576
Task completed at time 12.592674805771365
Task assigned to node with CPU power 4564 at time 13
Task completed at time 14.599035933391761
Task assigned to node with CPU power 4589 at time 15
Task assigned to node with CPU power 4255 at time 16
Task assigned to node with CPU power 4926 at time 17
Task completed at time 18.38483329701460